In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [2]:
df = pd.read_csv("big_startup_secsees_dataset.csv")
df.shape

(66368, 14)

In [3]:
df.head()

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,/organization/-fame,#fame,http://livfame.com,Media,10000000,operating,IND,16,Mumbai,Mumbai,1,NaN,2015-01-05,2015-01-05
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,3406878,operating,NaN,NaN,NaN,NaN,1,NaN,2014-01-30,2014-01-30
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   permalink          66368 non-null  object
 1   name               66367 non-null  object
 2   homepage_url       61310 non-null  object
 3   category_list      63220 non-null  object
 4   funding_total_usd  66368 non-null  object
 5   status             66368 non-null  object
 6   country_code       59410 non-null  object
 7   state_code         57821 non-null  object
 8   region             58338 non-null  object
 9   city               58340 non-null  object
 10  funding_rounds     66368 non-null  int64 
 11  founded_at         51147 non-null  object
 12  first_funding_at   66344 non-null  object
 13  last_funding_at    66368 non-null  object
dtypes: int64(1), object(13)
memory usage: 7.1+ MB


In [5]:
df.describe()

,funding_rounds
count,66368.000000
mean,1.732522
std,1.360251
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,19.000000


In [8]:
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,5058
category_list,3148
funding_total_usd,0
status,0
country_code,6958
state_code,8547
region,8030
city,8028


In [9]:
df = df.dropna(subset=['founded_at'])

In [12]:
df.loc[:, 'country_code'] = df['country_code'].fillna('Unknown')

In [13]:
df = df.dropna(subset=['category_list'])

In [15]:
df = df.dropna(subset=['name'])

In [20]:
df['funding_total_usd'] = (
    df['funding_total_usd']
    .replace('-', np.nan)
    .str.replace(',', '', regex=False)
)

df['funding_total_usd'] = pd.to_numeric(
    df['funding_total_usd'],
    errors='coerce'
)

In [21]:
df['funding_bucket'] = pd.cut(
    df['funding_total_usd'],
    bins=[0, 1e6, 1e7, 1e8, 1e9],
    labels=['Low', 'Medium', 'High', 'Very High']
)

In [17]:
df['first_funding_at'] = df['first_funding_at'].fillna(df['last_funding_at'])

In [18]:
df.isnull().sum()

,0
permalink,0
name,0
homepage_url,2499
category_list,0
funding_total_usd,0
status,0
country_code,0
state_code,4268
region,3802
city,3801


In [22]:
date_cols = ['founded_at', 'first_funding_at', 'last_funding_at']

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [23]:
df['failed'] = df['status'].apply(
    lambda x: 1 if x == 'closed' else 0
)

In [24]:
df['failed'].value_counts()

,count
failed,
0,45993
1,3717


In [25]:
current_year = pd.Timestamp.now().year

df['founded_year'] = df['founded_at'].dt.year
df['last_funding_year'] = df['last_funding_at'].dt.year

df['startup_age_years'] = np.where(
    df['failed'] == 1,
    df['last_funding_year'] - df['founded_year'],
    current_year - df['founded_year']
)

In [26]:
df = df[df['startup_age_years'] >= 0]

In [27]:
df['funding_rounds'] = pd.to_numeric(
    df['funding_rounds'],
    errors='coerce'
)

In [28]:
df['primary_category'] = df['category_list'].str.split('|').str[0]

In [29]:
upper_limit = df['funding_total_usd'].quantile(0.99)

df = df[df['funding_total_usd'] <= upper_limit]

In [30]:
df = df.dropna(subset=['founded_year', 'primary_category'])

In [31]:
df_clean = df[[
    'name',
    'primary_category',
    'country_code',
    'funding_total_usd',
    'funding_rounds',
    'status',
    'failed',
    'startup_age_years',
    'founded_year'
]]

df_clean.head()

,name,primary_category,country_code,funding_total_usd,funding_rounds,status,failed,startup_age_years,founded_year
1,:Qounter,Application Platforms,USA,700000.0,2,operating,0,12.0,2014.0
3,0-6.com,Curated Web,CHN,2000000.0,1,operating,0,19.0,2007.0
6,Ondine Biomedical Inc.,Biotechnology,CAN,762851.0,2,operating,0,29.0,1997.0
7,H2O.ai,Analytics,USA,33600000.0,4,operating,0,15.0,2011.0
8,One Inc.,Mobile,USA,1150050.0,3,operating,0,15.0,2011.0


In [32]:
df_clean.to_csv("clean_startup_data.csv", index=False)